In [1]:
def train_lstm_model(config, X_seq, y_seq):
    from torch.utils.data import DataLoader

    split_idx = int(len(X_seq) * 0.8)
    train_ds = StockDataset(X_seq[:split_idx], y_seq[:split_idx])
    val_ds = StockDataset(X_seq[split_idx:], y_seq[split_idx:])

    train_dl = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=config['batch_size'])

    model = LSTMRegressor(
        input_size=X_seq.shape[2],
        hidden_size=config['hidden_size'],
        num_layers=config['num_layers']
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    loss_fn = nn.MSELoss()

    for epoch in range(config['epochs']):
        model.train()
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Evaluate on validation set
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for xb, yb in val_dl:
            pred = model(xb)
            all_preds.append(pred)
            all_targets.append(yb)

    preds = torch.cat(all_preds).numpy()
    targets = torch.cat(all_targets).numpy()
    rmse = np.sqrt(np.mean((preds - targets) ** 2))
    return rmse, config


In [2]:
import random

param_space = {
    'SEQ_LEN': [10, 20, 30],
    'hidden_size': [32, 64, 128],
    'num_layers': [1, 2],
    'batch_size': [16, 32],
    'learning_rate': [0.001, 0.0005],
    'epochs': [20]
}

def sample_config():
    return {k: random.choice(v) for k, v in param_space.items()}

results = []
NUM_TRIALS = 10

for trial in range(NUM_TRIALS):
    config = sample_config()
    print(f"Trial {trial+1}/{NUM_TRIALS} with config: {config}")

    # Prepare new sequences for each SEQ_LEN
    SEQ_LEN = config['SEQ_LEN']
    X_seq, y_seq = [], []
    for i in range(len(data_scaled) - SEQ_LEN):
        X_seq.append(data_scaled[i:i+SEQ_LEN])
        y_seq.append(data_scaled[i+SEQ_LEN][-1])
    X_seq = np.array(X_seq, dtype=np.float32)
    y_seq = np.array(y_seq, dtype=np.float32)

    rmse, used_config = train_lstm_model(config, X_seq, y_seq)
    results.append((rmse, used_config))
    print(f"→ RMSE: {rmse:.4f}\n")

# Sort and show best config
results.sort(key=lambda x: x[0])
print("🏆 Best Configuration:")
print(results[0][1])
print(f"Lowest RMSE: {results[0][0]:.4f}")

Trial 1/10 with config: {'SEQ_LEN': 20, 'hidden_size': 32, 'num_layers': 1, 'batch_size': 32, 'learning_rate': 0.0005, 'epochs': 20}


NameError: name 'data_scaled' is not defined